Capas personalizadas --- 0:00 min
===

* Última modificación: Marzo 1, 2022 | YouTube

Importación de librerías
---

In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
import tensorflow_datasets as tfds

Conjuntos de operaciones comunes
----

In [2]:
layer = tf.keras.layers.Dense(100)

In [3]:
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

In [4]:
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [5]:
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.02325332,  0.41240185, -0.04722053,  0.00823265, -0.5001466 ,
         -0.17735547,  0.0699051 ,  0.54058   ,  0.35389382, -0.08745921],
        [ 0.3547942 ,  0.41461843, -0.5319194 , -0.2816233 , -0.2989134 ,
         -0.38525283, -0.3751296 , -0.26973662, -0.314858  , -0.41375172],
        [ 0.02956992, -0.08834106,  0.46280307,  0.42045122,  0.4816653 ,
         -0.14118266,  0.50701267, -0.23256588,  0.0253154 , -0.23714414],
        [ 0.14449716,  0.30955386, -0.21363354,  0.31867987,  0.18757445,
          0.0325762 , -0.2127009 ,  0.62799007, -0.1279037 ,  0.17440587],
        [ 0.5919631 ,  0.22138774, -0.43562776, -0.03065169, -0.45233214,
         -0.09552449,  0.09480822, -0.3697552 , -0.51259685,  0.23746729]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [6]:
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.02325332,  0.41240185, -0.04722053,  0.00823265, -0.5001466 ,
         -0.17735547,  0.0699051 ,  0.54058   ,  0.35389382, -0.08745921],
        [ 0.3547942 ,  0.41461843, -0.5319194 , -0.2816233 , -0.2989134 ,
         -0.38525283, -0.3751296 , -0.26973662, -0.314858  , -0.41375172],
        [ 0.02956992, -0.08834106,  0.46280307,  0.42045122,  0.4816653 ,
         -0.14118266,  0.50701267, -0.23256588,  0.0253154 , -0.23714414],
        [ 0.14449716,  0.30955386, -0.21363354,  0.31867987,  0.18757445,
          0.0325762 , -0.2127009 ,  0.62799007, -0.1279037 ,  0.17440587],
        [ 0.5919631 ,  0.22138774, -0.43562776, -0.03065169, -0.45233214,
         -0.09552449,  0.09480822, -0.3697552 , -0.51259685,  0.23746729]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

Implementación de capas personalizadas
---

In [7]:
class MyDenseLayer(tf.keras.layers.Layer):
    #
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs

    def build(self, input_shape):
        self.kernel = self.add_weight(
            "kernel", shape=[int(input_shape[-1]), self.num_outputs]
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)


layer = MyDenseLayer(10)

In [8]:
_ = layer(tf.zeros([10, 5]))

In [9]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']


Composición de capas en modelos
---

In [10]:
class ResnetIdentityBlock(tf.keras.Model):
    #
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name="")
        filters1, filters2, filters3 = filters

        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()

        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding="same")
        self.bn2b = tf.keras.layers.BatchNormalization()

        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2c(x)
        x = self.bn2c(x, training=training)

        x += input_tensor
        return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [11]:
_ = block(tf.zeros([1, 2, 3, 3]))

In [12]:
block.layers

In [13]:
len(block.variables)

18

In [14]:
block.summary()

Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  4         
                                                                 
 batch_normalization (BatchN  multiple                 4         
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           multiple                  4         
                                                                 
 batch_normalization_1 (Batc  multiple                 8         
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           multiple                  9         
                                                                 
 batch_normalization_2 (Batc  multiple                 12        


In [15]:
#
# Construcción de un modelo de capas apiladas
#
my_seq = tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(1, (1, 1), input_shape=(None, None, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(2, 1, padding="same"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(3, (1, 1)),
        tf.keras.layers.BatchNormalization(),
    ]
)
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [16]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, None, None, 1)     4         
                                                                 
 batch_normalization_3 (Batc  (None, None, None, 1)    4         
 hNormalization)                                                 
                                                                 
 conv2d_4 (Conv2D)           (None, None, None, 2)     4         
                                                                 
 batch_normalization_4 (Batc  (None, None, None, 2)    8         
 hNormalization)                                                 
                                                                 
 conv2d_5 (Conv2D)           (None, None, None, 3)     9         
                                                                 
 batch_normalization_5 (Batc  (None, None, None, 3)    1